In [3]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from IPython.display import clear_output

Al hacer la búsqueda y filtrar los tipos de propiedades a departamentos, casas y phs, me dí cuenta de que lo unico que variaba entre las páginas es el número al final, entonces hice una lista para luego iterarlos uno por uno y extraer los links a las 20 propiedades individuales que hay dentro de cada página.

In [5]:
urls_pages = []
for e in range(1, 406):
    urls_pages.append(f'https://www.argenprop.com/departamento-y-casa-y-ph-venta-partido-general-san-martin-pagina-{e}')

Como la primera vez que lo intente con la lista completa mi computadora dejo de responder a la mitad. Decidí separarlo en 4 chunks y comenzar todo el proceso de vuelta en cada chunk y no acumular excesiva memoria (No estoy seguro si esto tuvo mucho sentido pero finalmente funcionó).

In [15]:
chunks = (urls_pages[:100],urls_pages[100:200], urls_pages[200:300], urls_pages[300:405])

In [19]:
urls_global = []

for urls_pages in chunks:
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
    clear_output(wait=False)
    for url in urls_pages:
        driver.get(url)
        element = driver.find_element(By.CLASS_NAME ,'listing-container')
        elements = element.find_elements(By.CLASS_NAME, 'listing__item ')
        urls_local = []
        for e in elements:
            urls_local.append(e.find_element(By.ID, f'id-card-{elements.index(e) + 1}').get_attribute('href'))
        urls_global = urls_global + urls_local
        clear_output(wait=True)
        print(f'Chunk número {(chunks.index(urls_pages)) + 1} de {len(chunks)}')
        print(f'Página número {urls_pages.index(url) + 1} extraída ({((urls_pages.index(url) + 1)/len(urls_pages))*100}%)')
    driver.close()

Chunk número 4 de 4
Página número 105 extraída (100.0%)


In [20]:
len(urls_global)

8100

Finalmente tras extraer la url de las viviendas en venta en San Martín, proceso que duro aproximadamente 40 minutos, pasé la lista a un DataFrame de pandas para luego guardarlo como un archivo csv.

In [21]:
import pandas as pd

In [24]:
df = pd.DataFrame(urls_global)

In [25]:
df.to_csv('urls_propiedades_en_venta_san_martin_argenprop.csv')

In [28]:
df

,0
0,https://www.argenprop.com/departamento-en-vent...
1,https://www.argenprop.com/departamento-en-vent...
2,https://www.argenprop.com/departamento-en-vent...
3,https://www.argenprop.com/departamento-en-vent...
4,https://www.argenprop.com/departamento-en-vent...
...,...
8095,https://www.argenprop.com/departamento-en-vent...
8096,https://www.argenprop.com/departamento-en-vent...
8097,https://www.argenprop.com/departamento-en-vent...
8098,https://www.argenprop.com/departamento-en-vent...
